In [1]:
import pandas as pd
import yfinance as yf

df_tickers = pd.read_csv('../data/sp500.csv')
tickers = df_tickers['Symbol'].to_list()
tickers = [ticker.replace('.', '-') for ticker in tickers]
period = "max"
interval = "1m"
df = yf.download(tickers, period=period, interval=interval)['Adj Close']
print(df.head())
# df['date'] = df.index

In [3]:
df.info()

In [4]:
df.columns

In [3]:
from utils.StockClassifier import StockClassifier
import yfinance as yf
import pandas as pd

# Example usage
# Define the stock tickers and S&P 500 ticker
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Example tickers
sp500_ticker = '^GSPC'

# Download historical data for the stocks and the S&P 500 index
stock_data = yf.download(tickers, start='2020-01-01', end='2023-01-01')
sp500_data = yf.download(sp500_ticker, start='2020-01-01', end='2023-01-01')

# Close price and volume data
stock_df = stock_data['Close']
volume_df = stock_data['Volume']
sp500_df = sp500_data['Close']

print(stock_df.head())
print(volume_df.head())

# Initialize the StockClassifier
classifier = StockClassifier(stock_df, volume_df, sp500_df, tickers)

# Risk-based clustering
risk_clusters = classifier.risk_based_clustering()
print("Risk-based Clustering Results:")
print(risk_clusters)

# Performance-based clustering
performance_clusters = classifier.performance_clustering()
print("Performance-based Clustering Results:")
print(performance_clusters)

# Volume-based clustering
# volume_clusters = classifier.volume_based_clustering()
# print("Volume-based Clustering Results:")
# print(volume_clusters)

# Price-based clustering
# price_clusters = classifier.price_based_clustering()
# print("Price-based Clustering Results:")
# print(price_clusters)

# Example of fundamental data (e.g., P/E ratio, market cap)
fundamentals_data = pd.DataFrame({
    'Ticker': tickers,
    'P/E Ratio': [28.5, 35.2, 30.1],
    'Market Cap': [2.4e12, 1.8e12, 1.5e12]
}).set_index('Ticker')

# Fundamental-based clustering
fundamental_clusters = classifier.fundamental_based_clustering(fundamentals_data)
print("Fundamental-based Clustering Results:")
print(fundamental_clusters)

In [ ]:
print("Hello World")
import tensorflow as tf
import numpy as np
import gym

# Define the environment
env = gym.make('CartPole-v0')

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(24, input_shape=(4,), activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')
])


# Define the agent
class DQNAgent:
    def __init__(self, env, model):
        self.env = env
        self.model = model
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.batch_size = 32
        self.optimizer = tf.keras.optimizers.Adam(lr=0.001)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def choose_action(self, state):
        if np.random.rand() <= self.epsilon:
            return self.env.action_space.sample()
        else:
            return np.argmax(self.model.predict(state))

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        batch = np.random.choice(self.memory, self.batch_size, replace=False)
        states = np.array([i[0] for i in batch])
        actions = np.array([i[1] for i in batch])
        rewards = np.array([i[2] for i in batch])
        next_states = np.array([i[3] for i in batch])
        dones = np.array([i[4] for i in batch])

        # Calculate Q values
        target = rewards + self.gamma * np.amax(self.model.predict(next_states), axis=1)
        target[dones] = rewards[dones]

        # Update the model
        self.model.fit(states, target, batch_size=self.batch_size, verbose=0)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


# Initialize the agent
agent = DQNAgent(env, model)

# Train the agent
for episode in range(1000):
    state = env.reset()
    state = np.reshape(state, [1, 4])
    done = False
    score = 0
    while not done:
        # Choose action
        action = agent.choose_action(state)

        # Take action and observe next state and reward
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, [1, 4])

        # Remember the experience
        agent.remember(state, action, reward, next_state, done)

        # Update state
        state = next_state

        # Update score
        score += reward

    # Replay
    agent.replay()

    # Print score
    print("Episode: {}, Score: {}".format(episode, score))

# Test the agent
state = env.reset()
state = np.reshape(state, [1, 4])
done = False
score = 0
while not done:
    # Choose action
    action = agent.choose_action(state)

    # Take action and observe next state and reward
    next_state, reward, done, info = env.step(action)
    next_state = np.reshape(next_state, [1, 4])

    # Update state
    state = next_state

    # Update score
    score += reward

# Print final score
print("Final Score: {}".format(score))

# Close the environment
env.close()

In [6]:
import yfinance as yf

ticker = yf.Ticker('AAPL')
ticker.info

In [12]:

from datetime import datetime
from utils.StockDataManager import StockDataManager
import pandas as pd
import yfinance as yf
symbols = ["AAPL", "MSFT"]
period = '3mo'

stockDataManager = StockDataManager()

# Fetch historical data for the portfolio
portfolio_data_list = [stockDataManager.history(ticker=symbol, period=period, interval='1d')[['adj_close']] for symbol
                       in symbols]
portfolio_data = pd.concat(portfolio_data_list, axis=1, keys=[symbol for symbol in symbols])
portfolio_data.columns = portfolio_data.columns.droplevel(1)
portfolio_data = portfolio_data.ffill()
portfolio_data = portfolio_data.bfill()
mean_portfolio_data = portfolio_data.mean(axis=1)

# Convert dates to strings
mean_portfolio_data.index = mean_portfolio_data.index.astype(str)

# Fetch historical data for S&P 500
sp500_data = yf.download('^GSPC', start=datetime.strptime(portfolio_data.index.min(), '%Y-%m-%d').date(), end=datetime.strptime(portfolio_data.index.max(), '%Y-%m-%d').date(), interval='1d')['Adj Close']

# Convert dates to strings
sp500_data.index = sp500_data.index.astype(str)

In [14]:
datetime.strptime(portfolio_data.index.min(), '%Y-%m-%d').date()

In [12]:
from utils.StockDataManager import StockDataManager
stockDataManager = StockDataManager()

data = stockDataManager.history(ticker='AAPL', period='1y', interval='1d')
data

D:\College\Sem 2\Reinforcement Learning\Equity-Lens\utils\StockDataManager.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['dividends'] = data['dividends'].fillna(0)
D:\College\Sem 2\Reinforcement Learning\Equity-Lens\utils\StockDataManager.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['splits'] = data['splits'].fillna(0)


,open,high,low,close,adj_close,volume,dividends,splits,symbol
datetime,,,,,,,,,
2024-07-24 00:00:00,224.000000,224.800003,217.130005,218.539993,218.539993,61630300.0,0,0,AAPL
2024-07-23 00:00:00,224.369995,226.940002,222.679993,225.009995,225.009995,39960300.0,0,0,AAPL
2024-07-22 00:00:00,227.009995,227.779999,223.089996,223.960007,223.960007,48201800.0,0,0,AAPL
2024-07-19 00:00:00,224.820007,226.800003,223.279999,224.309998,224.309998,49151500.0,0,0,AAPL
2024-07-18 00:00:00,230.279999,230.440002,222.270004,224.179993,224.179993,66034600.0,0,0,AAPL
...,...,...,...,...,...,...,...,...,...
2023-08-04 00:00:00,185.520004,187.380005,181.919998,181.990005,181.028610,115799700.0,0,0,AAPL
2023-08-03 00:00:00,191.570007,192.369995,190.690002,191.169998,190.160110,61235200.0,0,0,AAPL
2023-08-02 00:00:00,195.039993,195.179993,191.850006,192.580002,191.562668,50389300.0,0,0,AAPL


In [1]:
import yfinance as yf

ticker = yf.Ticker('AAPL')
ticker.news

[{'uuid': '4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b',
  'title': 'Magnificent Seven Stocks: Nvidia Stock Reverses Lower; Tesla Races Higher',
  'publisher': "Investor's Business Daily",
  'link': 'https://finance.yahoo.com/m/4205eaa9-f620-3a0b-a81a-0e82c7c9fd0b/magnificent-seven-stocks%3A.html',
  'providerPublishTime': 1722265962,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/kV2Fc5Kz3y8hNbXlVxHRQg--~B/aD01NjU7dz0xMDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/d5bee44b5deef90ccff5d488feb6722d',
     'width': 1000,
     'height': 565,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/My3O.I831YydYsn.iMoUvw--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ibd.com/d5bee44b5deef90ccff5d488feb6722d',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['NVDA', 'AAPL', 'TSLA', 'META', 'MSFT']},
 {'uuid': '6aee9b07-79ab-3f89-8b2a-69c4db3cb6ea'